In [1]:
import pandas as pd
import re
df = pd.read_csv(r"D:\upenn大三下\denghui project\Quantitative-Trading-System\国内期货日交易数据163250758(仅供复旦大学使用)\FUT_Fdt.csv",usecols=[0,1,3,4,5,6,7,9,10],header=0,names=['Date','Code','Exchange','Open','High','Low','Close','Volume','Position'])
df['Type'] = df['Code'].apply(lambda x: re.search('[A-Za-z]+', x).group())

In [2]:
SH = df[df['Exchange']=='SHFE']
Metal = ['AU','AG','CU','AL','ZN','PB','NI','SN','SS','RB']
F_metal = SH[SH['Type'].isin(Metal)]
F_metal

,Date,Code,Exchange,Open,High,Low,Close,Volume,Position,Type
24,2020-01-02,AL2001,SHFE,14355.0,14425.0,14355.0,14410.0,5391.0,45835.0,AL
25,2020-01-02,CU2001,SHFE,49150.0,49210.0,49050.0,49110.0,12194.0,33020.0,CU
26,2020-01-02,ZN2001,SHFE,18030.0,18065.0,17985.0,17985.0,6244.0,14860.0,ZN
27,2020-01-02,PB2001,SHFE,15030.0,15070.0,14850.0,14870.0,1120.0,8170.0,PB
28,2020-01-02,NI2001,SHFE,110190.0,111170.0,109240.0,110430.0,2394.0,14814.0,NI
...,...,...,...,...,...,...,...,...,...,...
468599,2022-12-30,PB2312,SHFE,15960.0,15960.0,15960.0,15960.0,0.0,4.0,PB
468600,2022-12-30,RB2312,SHFE,4036.0,4079.0,4022.0,4074.0,13.0,7554.0,RB
468603,2022-12-30,SN2312,SHFE,211230.0,211230.0,211230.0,211230.0,0.0,79.0,SN
468605,2022-12-30,SS2312,SHFE,16210.0,16295.0,16205.0,16295.0,6.0,54.0,SS


In [3]:
# 按照合约代码（Code）对数据进行分组
grouped = F_metal.groupby('Type')

continuous_data = pd.DataFrame()

# 定义加权平均函数
def weighted_average(group):
    weighted_sum = pd.DataFrame()
    weighted_sum['open'] = pd.Series((group['Open'] * group['Position']).sum())
    weighted_sum['high'] = (group['High'] * group['Position']).sum()
    weighted_sum['low'] = (group['Low'] * group['Position']).sum()
    weighted_sum['close'] = (group['Close'] * group['Position']).sum()
    weighted_sum['volume'] = (group['Volume'] * group['Position']).sum()
    total_position = group['Position'].sum()
    # print(weighted_sum/total_position)
    return weighted_sum / total_position

for type, group in grouped:
    # 按照交易日期排序
    group = group.sort_values(by='Date')
    continuous_price = group.groupby('Date').apply(weighted_average).reset_index()
    
    # 创建连续合约数据
    continuous_price['type'] = type
    continuous_data = pd.concat([continuous_data, continuous_price])
    # continuous_data.append(continuous_price)

continuous_df = pd.DataFrame(continuous_data)
continuous_df
# print(continuous_df)

,Date,level_1,open,high,low,close,volume,type
0,2020-01-02,0,4386.239700,4388.566082,4242.438904,4359.137823,145879.341174,AG
1,2020-01-03,0,4367.468485,4467.466461,4366.908475,4437.251384,417515.570970,AG
2,2020-01-06,0,4443.371283,4535.712365,4409.296204,4503.210004,468613.287088,AG
3,2020-01-07,0,4516.287941,4522.968073,4389.713581,4428.622571,553131.222756,AG
4,2020-01-08,0,4416.118960,4596.771379,4410.830050,4520.339572,602534.847505,AG
...,...,...,...,...,...,...,...,...
723,2022-12-26,0,23205.969137,23561.833888,23169.253425,23514.158225,77592.140725,ZN
724,2022-12-27,0,23544.917124,23871.962336,23453.578017,23792.704065,56468.565687,ZN
725,2022-12-28,0,23800.450225,23957.900120,23653.988139,23947.684755,59622.404779,ZN
726,2022-12-29,0,23872.387722,23929.227123,23753.989853,23791.389486,43557.266402,ZN


In [4]:
from factor import factor
# continuous_df['ATR']=pd.Series()
factor_name = 'ATR'

groups = continuous_df.groupby('type')
for type, group in groups:
    ft = factor(group)
    getattr(ft, f'get_{factor_name}')()
    continuous_df.loc[continuous_df['type']==type,'ATR']=ft.get_df()['ATR']

continuous_df

,Date,level_1,open,high,low,close,volume,type,ATR
0,2020-01-02,0,4386.239700,4388.566082,4242.438904,4359.137823,145879.341174,AG,NaN
1,2020-01-03,0,4367.468485,4467.466461,4366.908475,4437.251384,417515.570970,AG,NaN
2,2020-01-06,0,4443.371283,4535.712365,4409.296204,4503.210004,468613.287088,AG,NaN
3,2020-01-07,0,4516.287941,4522.968073,4389.713581,4428.622571,553131.222756,AG,NaN
4,2020-01-08,0,4416.118960,4596.771379,4410.830050,4520.339572,602534.847505,AG,NaN
...,...,...,...,...,...,...,...,...,...
723,2022-12-26,0,23205.969137,23561.833888,23169.253425,23514.158225,77592.140725,ZN,408.935344
724,2022-12-27,0,23544.917124,23871.962336,23453.578017,23792.704065,56468.565687,ZN,409.610271
725,2022-12-28,0,23800.450225,23957.900120,23653.988139,23947.684755,59622.404779,ZN,402.060393
726,2022-12-29,0,23872.387722,23929.227123,23753.989853,23791.389486,43557.266402,ZN,387.177144


In [5]:
# continuous_df['date'] = pd.to_datetime(continuous_df['Date']).tz_localize('UTC')
continuous_df['Date'] = pd.to_datetime(continuous_df['Date'])

# Now, set the time zone for the 'date' column
continuous_df['Date'] = continuous_df['Date'].dt.tz_localize('UTC')

In [6]:
factors = continuous_df[['Date','type','ATR']].sort_values(by=['Date'],ascending=True)
factors.dropna(inplace=True)
factors.set_index(['Date','type'],inplace=True)
factors

ATR
Date                      type            
2020-01-22 00:00:00+00:00 CU    407.242856
                          SS    155.751145
                          PB    223.662759
                          ZN    245.969960
                          AG     96.381988
...                                    ...
2022-12-30 00:00:00+00:00 AU      3.187252
                          AL    258.373119
                          AG    108.398390
                          SS    301.198225
                          ZN    374.848215

[7140 rows x 1 columns]

In [7]:
price = continuous_df[['Date','type','close']].sort_values(by=['Date'],ascending=True)
price = price.pivot(index='Date', columns='type', values='close')
# price.set_index(['date'],inplace=True)
# 净值曲线cumprod，

price

type,AG,AL,AU,CU,NI,PB,RB,SN,SS,ZN
Date,,,,,,,,,,
2020-01-02 00:00:00+00:00,4359.137823,14090.414949,345.611601,49297.766206,111189.715376,14920.240864,3534.436725,136368.569430,13870.866101,17867.250229
2020-01-03 00:00:00+00:00,4437.251384,14058.134557,350.639252,48864.959415,107828.207566,14608.673325,3528.329892,136145.459421,13778.932009,17738.783376
2020-01-06 00:00:00+00:00,4503.210004,14081.246604,357.954570,48736.350283,108146.759668,14770.933699,3540.893365,135681.479066,13916.191662,18115.723380
2020-01-07 00:00:00+00:00,4428.622571,14052.824849,354.601382,48971.281072,109311.874326,14831.762896,3554.742267,135997.391393,13946.980692,18168.176934
2020-01-08 00:00:00+00:00,4520.339572,14110.560111,360.525722,48830.608576,109120.750921,14965.324028,3595.898192,136435.664553,13980.067932,18343.413552
...,...,...,...,...,...,...,...,...,...,...
2022-12-26 00:00:00+00:00,5340.819764,18647.437053,408.060593,65990.067501,218115.369132,15985.870761,4000.922770,197943.334774,16537.130356,23514.158225
2022-12-27 00:00:00+00:00,5404.368014,19079.519645,408.794369,66368.082822,226886.091376,16118.495503,4058.945993,205119.797508,16735.095583,23792.704065
2022-12-28 00:00:00+00:00,5373.984294,18778.520855,409.430894,66397.755665,227310.339821,16181.002470,4043.980844,209418.520313,16713.347144,23947.684755


In [8]:
import alphalens as al
# factors.index = pd.to_datetime(factors.index, format='%Y-%m-%d').tz_localize('UTC')
# price.index = pd.to_datetime(price.index, format='%Y-%m-%d').tz_localize('UTC')
analysis = al.utils.get_clean_factor_and_forward_returns(
    factors,
    price,
    quantiles=5,  # Number of quantiles to divide the factor into
    periods=(1, 5, 10),  # Lookahead periods for forward returns
)
analysis

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\alphalens\utils.py:319: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  delta_days = period_len.components.days - mode(days_diffs).mode[0]
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\alphalens\utils.py:319: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will chang

Dropped 1.4% entries from factor data: 1.4% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\alphalens\utils.py:167: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(quantile_calc, quantiles, bins, zero_aware, no_raise)


1D        5D       10D       factor  \
date                      asset                                              
2020-01-22 00:00:00+00:00 CU    -0.010504 -0.050461 -0.055904   407.242856   
                          SS    -0.010502 -0.042505 -0.028128   155.751145   
                          PB    -0.019760 -0.063570 -0.061393   223.662759   
                          ZN    -0.009261 -0.052172 -0.065212   245.969960   
                          AG     0.000327  0.004473 -0.000044    96.381988   
...                                   ...       ...       ...          ...   
2022-12-16 00:00:00+00:00 RB    -0.029318 -0.010377  0.014348    78.761929   
                          SS    -0.032157 -0.052080 -0.023268   339.304285   
                          AU     0.006377  0.010023  0.020362     3.366704   
                          SN    -0.018267  0.028731  0.107592  5885.564523   
                          AL    -0.010158 -0.005911 -0.001243   285.962530   

                                 factor_quantile  
date                      asset                   
2020-01-22 00:00:00+00:00 CU                   4  
                          SS                   3  
                          PB                   3  
                          ZN                   4  
                          AG                   2  
...                                          ...  
2022-12-16 00:00:00+00:00 RB                   1  
                          SS                   3  
                          AU                   1  
                          SN                   5  
                          AL                   3  

[7040 rows x 5 columns]

In [9]:
# tear_sheet_data = al.tears.create_returns_tear_sheet(analysis)

# # Access individual tables from the tear sheet data dictionary
# factor_returns_table = tear_sheet_data['returns']
# information_table = tear_sheet_data['information']
# factor_returns_table
df = pd.DataFrame()

for i in range(1,6):
    # print(group)
    group = analysis[analysis['factor_quantile']==i].groupby('date').mean()[['1D','factor_quantile']]
    df = pd.concat([df,group])
    # continuous_data = pd.concat([continuous_data, continuous_price])

# 净值曲线


df

,1D,factor_quantile
date,,
2020-01-22 00:00:00+00:00,-0.004673,1.0
2020-01-23 00:00:00+00:00,-0.024822,1.0
2020-02-03 00:00:00+00:00,0.009886,1.0
2020-02-04 00:00:00+00:00,-0.004313,1.0
2020-02-05 00:00:00+00:00,-0.000504,1.0
...,...,...
2022-12-12 00:00:00+00:00,0.025752,5.0
2022-12-13 00:00:00+00:00,-0.009277,5.0
2022-12-14 00:00:00+00:00,-0.007921,5.0


In [10]:
df = df.reset_index().sort_values(by=['factor_quantile','date'],ascending=True)
# df.set_index(['date','factor_quantile'],inplace=True)
df

,date,1D,factor_quantile
0,2020-01-22 00:00:00+00:00,-0.004673,1.0
1,2020-01-23 00:00:00+00:00,-0.024822,1.0
2,2020-02-03 00:00:00+00:00,0.009886,1.0
3,2020-02-04 00:00:00+00:00,-0.004313,1.0
4,2020-02-05 00:00:00+00:00,-0.000504,1.0
...,...,...,...
3515,2022-12-12 00:00:00+00:00,0.025752,5.0
3516,2022-12-13 00:00:00+00:00,-0.009277,5.0
3517,2022-12-14 00:00:00+00:00,-0.007921,5.0
3518,2022-12-15 00:00:00+00:00,-0.011480,5.0


In [11]:
df['1D'] = 1+df['1D']
df['1D']=df[['1D','factor_quantile']].groupby('factor_quantile').cumprod()
df

,date,1D,factor_quantile
0,2020-01-22 00:00:00+00:00,0.995327,1.0
1,2020-01-23 00:00:00+00:00,0.970622,1.0
2,2020-02-03 00:00:00+00:00,0.980217,1.0
3,2020-02-04 00:00:00+00:00,0.975990,1.0
4,2020-02-05 00:00:00+00:00,0.975497,1.0
...,...,...,...
3515,2022-12-12 00:00:00+00:00,1.775499,5.0
3516,2022-12-13 00:00:00+00:00,1.759028,5.0
3517,2022-12-14 00:00:00+00:00,1.745094,5.0
3518,2022-12-15 00:00:00+00:00,1.725061,5.0
